# Zagadki

![image-2.png](attachment:image-2.png)

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to zadanie możesz zdobyć pomiędzy 0 i 1.5 punktu. Zdobędziesz 0 punktów jeśli wartość kryterium `mean reciprocal rank` na zbiorze testowym wyniesie poniżej 0.02, a 1.5 punktu jeśli wyniesie powyżej 0.3. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością kryterium.

# Kod startowy

In [ ]:
!pip install sacremoses

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
from torch.nn import functional as F
import random

## Ładowanie danych

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
path_to_data = 'data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)
    if ret:
        return ret
    return word

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
model = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')
    L = word.split()
    if len(L) == 1:
        word = L[0]

        definition = set(tokenize(definition.lower()))
        all_word_definitions[word].append(definition)
        for word in set(definition):
            base_idf[get_word_base(word)] += 1


for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()
        answers.append(line[0])
        queries.append(tokenize(' '.join(line[1:])))

## Kod z kryteriami oceniającymi

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)

    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)

    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [ ]:
model_name = "allegro/herbert-base-cased"
device = 'cuda'

tokenizer_herbert = AutoTokenizer.from_pretrained(model_name)
model_herbert = AutoModel.from_pretrained(model_name).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer_herbert(text, return_tensors='pt')['input_ids'][0]

print ([tokenizer_herbert.decode(idx) for idx in token_ids])

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['<s>', 'Bardzo', 'lubię', 'lody', 'mali', 'nowe', 'z', 'bi', 'tą', 'śmietan', 'ą', '.', '</s>']


In [ ]:
model_name_polka = "eryk-mazus/polka-1.1b-chat"

tokenizer_polka = AutoTokenizer.from_pretrained(model_name_polka)
model_polka = AutoModelForCausalLM.from_pretrained(model_name_polka).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label


def sentence_prob(sentence_txt):
    input_ids = tokenizer_polka(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model_polka(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()

In [ ]:
# data to use
# bases -  Dictionary mapping words to their base words
# all_word_definitions - word definitions
# base_idf = dd(int) - Dictionary containing all base words inverse document frequency


In [ ]:
THRESHOLD_IDF = 2.89
FORBIDDEN_WORDS = ["jednak", "lub", "przecież", "ależ", "one", "poza", "bardzo", "albo", "dany", "u", "przy", "ze", "z", "do", "jako", "jak", "także", "oraz"]
FIXED_TENSOR = torch.rand(768).to(device)


In [ ]:
def tensor_to_number(tensor):
    return torch.dot(tensor, FIXED_TENSOR).item()

def find_closest_key(d, query_float):
    return min(d.keys(), key=lambda k: abs(k - query_float))

In [ ]:
def prune_idf(l):
    return list(filter(lambda x: base_idf[bases.get(x)] > THRESHOLD_IDF and x not in FORBIDDEN_WORDS, l))

def word_to_embedding(word, tokenizer = tokenizer_herbert, model = model_herbert):

    input_ids = tokenizer(word, return_tensors='pt')['input_ids'].to(device)
    embeddings = model.embeddings.word_embeddings(input_ids)[0, :, :].mean(dim=0).detach()
    return embeddings

def list_of_words_to_embedding(word_list, tokenizer = tokenizer_herbert, model = model_herbert):
    input_ids = tokenizer(word_list, return_tensors='pt', padding=True)['input_ids'].to(device)

def create_embeddings(unique_words = False):
    all_embeddings = []
    d = {}
    for word, def_list in all_word_definitions.items():
        pruned_defs = list(map(prune_idf, def_list))
        new_def = [item for sublist in pruned_defs for item in sublist]
        if unique_words:
            new_def = list(set(new_def))
        try:
            embeddings = [word_to_embedding(s) for s in new_def]
            if len(embeddings) == 0:
                embeddings = [word_to_embedding(word)]
            embeddings_tensor = torch.stack(embeddings)
            mean_embedding = torch.mean(embeddings_tensor, dim=0)
            all_embeddings.append(mean_embedding)
            d[tensor_to_number(mean_embedding)] = word
        except RuntimeError:
            print(word)
            print(def_list)
            print(new_def)
    return torch.stack(all_embeddings), d

In [ ]:
definitions_embeddings_herbert, embedding_to_word = create_embeddings()

In [ ]:
definitions_embeddings_herbert.shape

torch.Size([8085, 768])

In [ ]:
for k, v in embedding_to_word.items():
    new_k = find_closest_key(embedding_to_word, k)
    new_v = embedding_to_word[new_k]
    if v != new_v:
        print(v, new_v)

In [ ]:
embedding_to_word.get(definitions_embeddings_herbert[0])

In [ ]:
def k_nearest_neighbours(k, all_tensors, query_tensor):
    distances = torch.norm(all_tensors - query_tensor, dim=1)  # dim=1 computes distance row-wise
    k_neighbors = torch.topk(distances, k, largest=False)  # `largest=False` ensures smallest distances
    nearest_def_embeddings = definitions_embeddings_herbert[k_neighbors.indices]
    return [embedding_to_word[tensor_to_number(t)] for t in nearest_def_embeddings]

In [ ]:
# all_word_definitions["zamek"]
len(definitions_embeddings_herbert.shape)

2

In [ ]:
def answer_riddle(riddle, K):
    # print(riddle)
    # print(random.sample(all_word_definitions.keys(), K))
    pruned_q = prune_idf(riddle)
    try:3149
        q_embeddings = [word_to_embedding(s) for s in pruned_q]
        if len(q_embeddings) == 0:
            q_embeddings = [torch.rand(768).to(device)]
        q_embeddings_tensor = torch.stack(q_embeddings)
        q_embedding = torch.mean(q_embeddings_tensor, dim=0)
    except RuntimeError:
        print(riddle)
    all_answers = k_nearest_neighbours(2 * K, definitions_embeddings_herbert, q_embedding)
    all_answers = list(map(lambda x: (x, sentence_prob(f"{riddle} {x}")), all_answers))
    print(all_answers)
    all_answers = sorted(all_answers, key=lambda x: x[1], reverse=True)
    all_answers = list(map(lambda x: x[0], all_answers))
    print(all_answers[:K])
    return all_answers[:K]

# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    for query in tqdm(queries, desc="queries answered"):
        computed_answers.append(score_function(set(query), K=K))
    score = mean_reciprocal_rank(answers, computed_answers, K=K)
    return score

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
# k nearest neighbours approach
if not FINAL_EVALUATION_MODE:
    # PART_OF_DATA =
    PART_OF_DATA = 293
    K = 20
    valid_queries = queries[:PART_OF_DATA]
    valid_answers = answers[:PART_OF_DATA]
    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered:   0%|          | 1/293 [00:02<10:24,  2.14s/it]

[('publikacja', array(-138.31998, dtype=float32)), ('manuskrypt', array(-140.66412, dtype=float32)), ('maszynopis', array(-144.61705, dtype=float32)), ('rękopis', array(-139.94963, dtype=float32)), ('papier', array(-139.28928, dtype=float32)), ('czcionka', array(-144.6078, dtype=float32)), ('spis', array(-140.0911, dtype=float32)), ('papierek', array(-150.27306, dtype=float32)), ('zapotrzebowanie', array(-145.0005, dtype=float32)), ('pismo', array(-142.31363, dtype=float32)), ('podpis', array(-138.42601, dtype=float32)), ('nagłówek', array(-139.62465, dtype=float32)), ('druk', array(-141.46068, dtype=float32)), ('dekret', array(-144.23122, dtype=float32)), ('artykuł', array(-139.75249, dtype=float32)), ('akt', array(-140.9214, dtype=float32)), ('książka', array(-140.97115, dtype=float32)), ('poemat', array(-144.78188, dtype=float32)), ('paragraf', array(-142.49913, dtype=float32)), ('patent', array(-142.76836, dtype=float32)), ('protokół', array(-146.13661, dtype=float32)), ('powtórzen

queries answered:   1%|          | 2/293 [00:04<10:58,  2.26s/it]

[('zimny', array(-193.08572, dtype=float32)), ('romantyzm', array(-195.57452, dtype=float32)), ('błąd', array(-194.47372, dtype=float32)), ('zwarcie', array(-196.08638, dtype=float32)), ('modernizm', array(-195.87767, dtype=float32)), ('ekspres', array(-192.74219, dtype=float32)), ('moda', array(-190.24709, dtype=float32)), ('zając', array(-194.16747, dtype=float32)), ('naturalizm', array(-197.25764, dtype=float32)), ('język', array(-188.12494, dtype=float32)), ('przewrót', array(-196.83133, dtype=float32)), ('konstytucja', array(-195.82205, dtype=float32)), ('piła', array(-195.42827, dtype=float32)), ('dźwignia', array(-196.3826, dtype=float32)), ('nalot', array(-194.32889, dtype=float32)), ('pełny', array(-191.75014, dtype=float32)), ('as', array(-188.74292, dtype=float32)), ('węzeł', array(-193.20947, dtype=float32)), ('fala', array(-191.4733, dtype=float32)), ('żółw', array(-195.99695, dtype=float32)), ('dżungla', array(-198.40869, dtype=float32)), ('ekspozycja', array(-194.86713, 

queries answered:   1%|          | 3/293 [00:08<14:19,  2.96s/it]

...

[('topnienie', array(-182.63895, dtype=float32)), ('nalot', array(-187.02737, dtype=float32)), ('zjazd', array(-188.62398, dtype=float32)), ('dźwignia', array(-188.34622, dtype=float32)), ('przewód', array(-186.99997, dtype=float32)), ('reakcja', array(-187.44502, dtype=float32)), ('syfon', array(-187.51218, dtype=float32)), ('filtr', array(-184.58833, dtype=float32)), ('spadochron', array(-186.38644, dtype=float32)), ('erozja', array(-189.45728, dtype=float32)), ('piła', array(-190.04013, dtype=float32)), ('absorpcja', array(-188.74504, dtype=float32)), ('rząd', array(-185.00243, dtype=float32)), ('zwarcie', array(-187.20221, dtype=float32)), ('intuicja', array(-188.19894, dtype=float32)), ('ekspozycja', array(-187.39342, dtype=float32)), ('podjazd', array(-189.16647, dtype=float32)), ('piractwo', array(-190.92879, dtype=float32)), ('gnicie', array(-192.49307, dtype=float32)), ('węzeł', array(-188.10963, dtype=float32)), ('kompensacja', array(-189.97171, dtype=float32)), ('indeks', ar

queries answered: 100%|█████████▉| 292/293 [15:08<00:02,  2.95s/it]

[('prototyp', array(-165.93478, dtype=float32)), ('zaopatrzenie', array(-162.04004, dtype=float32)), ('kolekcja', array(-161.62357, dtype=float32)), ('technika', array(-161.39674, dtype=float32)), ('wytrzymałość', array(-160.42737, dtype=float32)), ('otoczenie', array(-163.67517, dtype=float32)), ('skład', array(-155.79066, dtype=float32)), ('zbiornik', array(-164.28223, dtype=float32)), ('serce', array(-164.72093, dtype=float32)), ('adaptacja', array(-162.22481, dtype=float32)), ('etat', array(-166.68533, dtype=float32)), ('agregat', array(-163.69363, dtype=float32)), ('obyczaj', array(-165.17715, dtype=float32)), ('gąbka', array(-164.82059, dtype=float32)), ('unia', array(-163.8503, dtype=float32)), ('inwestycja', array(-163.38411, dtype=float32)), ('osobliwość', array(-167.20496, dtype=float32)), ('mechanizm', array(-161.81837, dtype=float32)), ('natrysk', array(-162.90137, dtype=float32)), ('rurociąg', array(-163.03395, dtype=float32)), ('portfel', array(-164.43515, dtype=float32))

queries answered: 100%|██████████| 293/293 [15:11<00:00,  3.11s/it]

[('obręb', array(-201.55656, dtype=float32)), ('domena', array(-199.34445, dtype=float32)), ('granica', array(-196.51227, dtype=float32)), ('luz', array(-198.9827, dtype=float32)), ('pustka', array(-197.5766, dtype=float32)), ('niż', array(-194.7496, dtype=float32)), ('teren', array(-196.47244, dtype=float32)), ('podjazd', array(-201.11053, dtype=float32)), ('sonda', array(-198.79529, dtype=float32)), ('region', array(-194.47618, dtype=float32)), ('wyż', array(-197.26753, dtype=float32)), ('sito', array(-200.56311, dtype=float32)), ('pozycja', array(-194.71548, dtype=float32)), ('obszar', array(-194.10397, dtype=float32)), ('drogowskaz', array(-201.08047, dtype=float32)), ('horyzont', array(-196.2127, dtype=float32)), ('tunel', array(-196.48792, dtype=float32)), ('łańcuch', array(-194.14606, dtype=float32)), ('plenum', array(-206.90335, dtype=float32)), ('galeria', array(-195.70348, dtype=float32)), ('przestrzeń', array(-193.87244, dtype=float32)), ('masyw', array(-201.61996, dtype=flo

In [ ]:
model_name = "eryk-mazus/polka-1.1b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
).to(device)

# Lista legalnych słów
legal_words = all_word_definitions.keys()
legal_token_ids = [tokenizer.encode(word, add_special_tokens=False)[0] for word in legal_words]

In [ ]:
for i, q in enumerate(queries):
    if i == 10:
      break
    q = " ".join(prune_idf(q))
    queries[i] = q
    print(q)
    print(answers[i])
    print(prune_idf(q))
    print()


manuskrypt
[]

a a a a a a a
wesołość
['a', 'a', 'a', 'a', 'a', 'a', 'a']

a a a a a a a a a a a a a
legenda
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']

a a a a a a a a a a a a
antysemityzm
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']

a a a a a
filmowanie
['a', 'a', 'a', 'a', 'a']

a a a a a a a a
nurkowanie
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']

a a a a a a a a
nonsens
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']

a a a a a a a a a
weto
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']

a a a a a a
mówca
['a', 'a', 'a', 'a', 'a', 'a']

a a a a a a a
sądownictwo
['a', 'a', 'a', 'a', 'a', 'a', 'a']



In [ ]:
q = queries[2]
prune_idf(q)
# q = " ".join(prune_idf(q))
# q

['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']

In [ ]:
# Funkcja generująca tylko legalne słowo
def generate_only_legal_word(model, tokenizer, legal_token_ids):
    input_ids = tokenizer.encode(q, return_tensors="pt").to(device)  # Pusty prompt

    while True:
        # Generowanie ciągu tokenów, który może zawierać więcej niż jeden token
        output = model.generate(
            input_ids,
            max_length=9 + 5,  # Dopuszczamy, by wygenerowane słowo składało się z maksymalnie 4 tokenów
            do_sample=True,
            top_k=50,  # Ograniczenie do najlepszych tokenów
            top_p=0.9,  # Ograniczenie do najbardziej prawdopodobnych tokenów
            temperature=0.7,  # Mniej chaotyczne generowanie
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

        # Dekodowanie wygenerowanych tokenów
        generated_tokens = output[0, len(input_ids[0]):]  # Otrzymujemy tokeny wygenerowane po prompt
        generated_word = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        w = bases.get(generated_word.split(" ")[0])
        print(w)

        # Sprawdzanie, czy wygenerowane słowo jest na liście legalnych słów
        if w in legal_words:
            return w  # Jeśli tak, zwracamy to słowo


In [ ]:
"czapka" in legal_words

True

In [ ]:
# Test
legal_word = generate_only_legal_word(model, tokenizer, legal_token_ids)
print(f"Wygenerowane legalne słowo: {legal_word}")

None
None
None
None
None
z
None
i
wers
Wygenerowane legalne słowo: wers
